## Import and install


In [1]:
!pip install scikit-learn==0.22.2  -q # to solve "AttributeError: 'CRF' object has no attribute 'keep_tempfiles'" when using crfsuite
!pip install sklearn_crfsuite -q
!pip install eli5 -q

In [2]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [3]:
import sys
sys.path.append('/content/drive/My Drive/DTA/Internship/')

In [4]:
from sklearn.linear_model import PassiveAggressiveClassifier, SGDClassifier, Perceptron
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction import DictVectorizer
from sklearn.model_selection import train_test_split
from collections import Counter
from process_for_baselines import process_data, test_classifier
from ner_crfsuite import train_crf, flat_classification_report
import pandas as pd
import numpy as np

# CLASSIFIERS

In [5]:
_, classes, new_classes, X_train, X_test, y_train, y_test = process_data("drive/MyDrive/DTA/Internship/C14NL.csv")
test_classifier(X_train, y_train, X_test, y_test, classes, new_classes, PassiveAggressiveClassifier) 

PassiveAggressiveClassifier(random_state=42)
              precision    recall  f1-score   support

      B-DATE       0.90      0.63      0.74        70
       B-LOC       0.80      0.62      0.70       164
     B-MONEY       0.46      0.51      0.48        71
      B-PERS       0.94      0.67      0.79       432
      I-DATE       0.57      0.23      0.33       288
       I-LOC       1.00      0.30      0.47        33
     I-MONEY       0.78      0.76      0.77       197
      I-PERS       0.47      0.27      0.34       410

   micro avg       0.72      0.49      0.58      1665
   macro avg       0.74      0.50      0.58      1665
weighted avg       0.71      0.49      0.57      1665



In [6]:
test_classifier(X_train, y_train, X_test, y_test, classes, new_classes, MultinomialNB, alpha=0.01) 

MultinomialNB(alpha=0.01)
              precision    recall  f1-score   support

      B-DATE       0.31      0.64      0.42        70
       B-LOC       0.54      0.68      0.60       164
     B-MONEY       0.24      0.54      0.33        71
      B-PERS       0.79      0.80      0.80       432
      I-DATE       0.53      0.33      0.40       288
       I-LOC       0.17      0.52      0.25        33
     I-MONEY       0.57      0.82      0.67       197
      I-PERS       0.57      0.35      0.44       410

   micro avg       0.54      0.58      0.56      1665
   macro avg       0.46      0.58      0.49      1665
weighted avg       0.58      0.58      0.56      1665



In [7]:
test_classifier(X_train, y_train, X_test, y_test, classes, new_classes, Perceptron, n_jobs=-1, max_iter=5)

Perceptron(max_iter=5, n_jobs=-1, random_state=42)
              precision    recall  f1-score   support

      B-DATE       0.90      0.61      0.73        70
       B-LOC       0.64      0.65      0.65       164
     B-MONEY       0.09      0.80      0.16        71
      B-PERS       0.94      0.71      0.81       432
      I-DATE       0.53      0.30      0.38       288
       I-LOC       1.00      0.24      0.39        33
     I-MONEY       0.59      0.79      0.68       197
      I-PERS       0.41      0.27      0.32       410

   micro avg       0.47      0.52      0.49      1665
   macro avg       0.64      0.55      0.52      1665
weighted avg       0.63      0.52      0.54      1665



In [8]:
test_classifier(X_train, y_train, X_test, y_test, classes, new_classes, SGDClassifier)

SGDClassifier(random_state=42)
              precision    recall  f1-score   support

      B-DATE       0.89      0.59      0.71        70
       B-LOC       0.86      0.45      0.59       164
     B-MONEY       0.47      0.30      0.36        71
      B-PERS       0.93      0.56      0.70       432
      I-DATE       0.72      0.22      0.34       288
       I-LOC       1.00      0.21      0.35        33
     I-MONEY       0.87      0.71      0.78       197
      I-PERS       0.90      0.09      0.17       410

   micro avg       0.85      0.37      0.52      1665
   macro avg       0.83      0.39      0.50      1665
weighted avg       0.85      0.37      0.48      1665



# CRF SUITE

In [5]:
df, _, new_classes, _, _, _, _ = process_data("drive/MyDrive/DTA/Internship/C14NL.csv")

In [6]:
crf, _, X_test, _, y_test = train_crf(df)
y_pred = crf.predict(X_test)
print(flat_classification_report(y_test, y_pred, labels = new_classes))

              precision    recall  f1-score   support

      B-DATE       0.87      0.86      0.87       129
       B-LOC       0.89      0.78      0.83       235
     B-MONEY       0.84      0.83      0.84       119
      B-PERS       0.94      0.82      0.88       679
      I-DATE       0.96      0.90      0.93       488
       I-LOC       0.90      0.61      0.73        44
     I-MONEY       0.84      0.86      0.85       330
      I-PERS       0.93      0.84      0.88       604

   micro avg       0.91      0.84      0.88      2628
   macro avg       0.90      0.81      0.85      2628
weighted avg       0.92      0.84      0.88      2628



In [15]:
def print_transitions(trans_features):
    for (label_from, label_to), weight in trans_features:
        print("%-6s -> %-7s %0.6f" % (label_from, label_to, weight))
print("Top likely transitions:")
print_transitions(Counter(crf.transition_features_).most_common(20))
print("\nTop unlikely transitions:")
print_transitions(Counter(crf.transition_features_).most_common()[-20:])

Top likely transitions:
B-PERS -> I-PERS  0.451502
I-DATE -> I-DATE  0.437716
B-DATE -> I-DATE  0.385789
B-MONEY -> I-MONEY 0.364858
I-MONEY -> I-MONEY 0.363493
I-PERS -> I-PERS  0.293811
O      -> B-LOC   0.275824
O      -> O       0.272508
B-LOC  -> I-LOC   0.265046
O      -> B-PERS  0.241094
I-LOC  -> I-LOC   0.220843
O      -> B-DATE  0.204829
O      -> B-MONEY 0.171163
I-PERS -> B-PERS  0.113904
B-LOC  -> O       0.056318
I-PERS -> O       0.040300
B-PERS -> O       0.033679
B-DATE -> O       0.029255
I-DATE -> B-DATE  0.028593
B-LOC  -> B-MONEY 0.020041

Top unlikely transitions:
B-LOC  -> I-PERS  -0.082857
I-PERS -> I-LOC   -0.084592
B-LOC  -> I-DATE  -0.087840
I-DATE -> B-LOC   -0.102097
I-PERS -> B-LOC   -0.105342
O      -> I-LOC   -0.112765
I-DATE -> B-PERS  -0.114827
B-PERS -> I-MONEY -0.118562
B-PERS -> I-DATE  -0.124122
I-DATE -> B-MONEY -0.133441
I-PERS -> I-MONEY -0.136772
I-DATE -> O       -0.137457
I-DATE -> I-PERS  -0.139042
I-DATE -> I-MONEY -0.140636
I-MONEY -> I-PE

- The most likely transition understandably includes several instances of B-class -> I-class for the same class. 
- Several I-class -> I-class (same class)transitions also appear to be very common (such as I-DATE -> I-DATE, which is the second most likely transition).
- O -> B-class of all classes are also very likely transitions
- O -> I-class of all classes are understandably the least likely transitions (same as the English dataset)
- I-PERS -> B-PERS is surprisingly likely, perhaps our dataset includes a lot of enumerations of personal names

In [16]:
def print_state_features(state_features):
    for (attr, label), weight in state_features:
        print("%0.6f %-8s %s" % (weight, label, attr))
print("Top positive:")
print_state_features(Counter(crf.state_features_).most_common(30))
print("\nTop negative:")
print_state_features(Counter(crf.state_features_).most_common()[-30:])

Top positive:
0.573776 B-LOC    word.lower():auden
0.527893 B-PERS   word.lower():symoene
0.503007 O        word.lower():hem
0.494820 B-DATE   -1:word.lower():screef
0.487776 I-PERS   -1:word.lower():vander
0.474440 B-PERS   word.lower():marien
0.473987 B-PERS   word.lower():janne
0.465352 B-DATE   word.lower():kersauonde
0.456608 I-PERS   +1:word.lower():daniel
0.455877 B-PERS   word.lower():pietren
0.453020 O        word.lower():voers
0.448215 B-PERS   word.lower():jonghe
0.426829 B-PERS   word.lower():pieter
0.426577 B-DATE   word.lower():midden
0.421016 B-PERS   word.lower():katelinen
0.420281 I-DATE   -1:word.lower():.m.ccc.
0.420104 B-LOC    word.lower():velseke
0.414966 O        word.lower():sine
0.412633 I-PERS   -1:word.lower():der
0.408862 B-LOC    +1:word.lower():ambochte
0.407218 B-PERS   word.lower():gode
0.404432 B-PERS   word.lower():piet
0.399935 I-PERS   -1:word.lower():den
0.396202 B-PERS   word.lower():gillise
0.395444 I-DATE   -1:word.lower():midden
0.394233 B-LOC  

Compared to the English dataset:
- The "BOS" feature isn't in the top positive list, possibly because the sentences are so much longer so there are less instances of beginning of sentence.
- The few words before a certain word seem to have less influence on its tagging, especially if it's more than 1 word before (in the English dataset there are quite a few instances of "word[-3]" in the top positive
- The top negative does include several instances of "word[-2 or 3]" 

In [7]:
import eli5
eli5.show_weights(crf, top=10)

/usr/local/lib/python3.8/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.feature_selection.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.feature_selection. Anything that cannot be imported from sklearn.feature_selection is now part of the private API.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.8/dist-packages/sklearn/base.py:193: FutureWarning: From version 0.24, get_params will raise an AttributeError if a parameter cannot be retrieved as an instance attribute. Previously it would return None.
  warnings.warn('From version 0.24, get_params will raise an '


- For the English dataset, the third highest feature for O is BOS, here it's not in the top 10 features for O, possibly due to sentence length, as mentioned previously.
- However, for I-DATE, EOS is the 4th highest feature. This is because the Middle Dutch dataset is divided by charters as opposed to sentences, and the charters almost always end in dates.


In [8]:
eli5.show_weights(crf, top=10, targets=['O', 'B-DATE', 'I-PERS'])

In [9]:
eli5.show_weights(crf, top=10, feature_re='^word\.is',
                  horizontal_layout=False, show=['targets'])

Weight?,Feature
+0.140,word.isupper()
+0.012,word.istitle()
Weight?,Feature
-0.004,word.isupper()
-0.033,word.istitle()
Weight?,Feature
-0.066,word.isupper()
-0.141,word.istitle()
Weight?,Feature
+0.086,word.istitle()


- As opposed to the English dataset, words being in capital letters increases their chance of being "O" in the Dutch dataset
- Both isupper() and istitle() are top negative features for dates (B and I)
- Understandably, personal names are often istitle(), as personal always start with a capital letter